In [74]:
# READ: A GTFS data acquisition and processing framework and its application to train delay prediction
# https://www.sciencedirect.com/science/article/pii/S2046043022000090

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

routes = pd.read_csv('../datasets/routes.txt', sep=',')
trips = pd.read_csv('../datasets/trips.txt', sep=',')
stops = pd.read_csv('../datasets/stops.txt', sep=',')
stop_times = pd.read_csv('../datasets/stop_times.txt', sep=',')

route_id = 121

In [75]:
# 1. Filter trips for the chosen route_id
route_trips = trips[trips['route_id'] == route_id]

route_trips

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,wheelchair_accessible,note_fr,note_en
55588,121,25J-H58J000A-81-A,281568639,Est,0,1210266,1,NaN,NaN
55590,121,25J-H58J000A-81-A,281568642,Ouest,1,1210271,1,NaN,NaN
55591,121,25J-H58J000A-81-A,281568643,Est,0,1210266,1,NaN,NaN
55593,121,25J-H58J000A-81-A,281568645,Ouest,1,1210271,1,NaN,NaN
55603,121,25J-H58J000A-81-A,281568655,Ouest,1,1210271,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
151082,121,25M-H58M300F-80-F3,284745720,Est,0,1210266,1,NaN,NaN
151083,121,25M-H58M300F-80-F3,284745721,Ouest,1,1210271,1,NaN,NaN
151084,121,25M-H58M300F-80-F3,284745722,Est,0,1210266,1,NaN,NaN
151085,121,25M-H58M300F-80-F3,284745723,Ouest,1,1210271,1,NaN,NaN


In [76]:
# 2. Get the list of trip_ids for this route
trip_ids = route_trips['trip_id'].unique()


trip_ids

array([281568639, 281568642, 281568643, ..., 284745722, 284745723,
       284745724])

In [77]:
# 3. Find all stop_times entries corresponding to these trip_ids
route_stop_times = stop_times[stop_times['trip_id'].isin(trip_ids)]

route_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
1933432,281568639,07:02:00,07:02:00,55934,1
1933433,281568639,07:02:26,07:02:26,55936,2
1933434,281568639,07:04:00,07:04:00,55349,3
1933435,281568639,07:04:46,07:04:46,55454,4
1933436,281568639,07:05:59,07:05:59,55892,5
...,...,...,...,...,...
5293818,284745724,20:23:19,20:23:19,50634,44
5293819,284745724,20:23:51,20:23:51,50667,45
5293820,284745724,20:24:26,20:24:26,50686,46
5293821,284745724,20:25:00,20:25:00,50719,47


In [92]:
# Get unique stop_ids for the route (to avoid duplicates if the same stop is used in multiple trips)
stop_ids = route_stop_times['stop_id'].unique()

stop_ids = list(map(str, stop_ids))

In [103]:
# # 4. Filter stops for these stop_ids to get details like latitude and longitude
route_stops = stops[stops['stop_id'].isin(stop_ids)]

route_stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,location_type,parent_station,wheelchair_boarding
1028,50325,50325,Sauvé / de l'Acadie (côté est),45.535882,-73.665750,https://www.stm.info/fr/recherche#stq=50325,0,NaN,1
1030,50328,50328,Sauvé / de l'Acadie,45.536011,-73.665364,https://www.stm.info/fr/recherche#stq=50328,0,NaN,1
1032,50330,50330,Sauvé / de l'Acadie (Côté ouest),45.535195,-73.665883,https://www.stm.info/fr/recherche#stq=50330,0,NaN,1
1042,50341,50341,Sauvé / Meilleur,45.540450,-73.664312,https://www.stm.info/fr/recherche#stq=50341,0,NaN,1
1043,50342,50342,Sauvé / Meilleur,45.540662,-73.663885,https://www.stm.info/fr/recherche#stq=50342,0,NaN,1
...,...,...,...,...,...,...,...,...,...
6275,55865,55865,Côte-Vertu / No 100,45.529531,-73.668314,https://www.stm.info/fr/recherche#stq=55865,0,NaN,1
6301,55892,55892,Cavendish / Place Vertu,45.500335,-73.704089,https://www.stm.info/fr/recherche#stq=55892,0,NaN,1
6307,55898,55898,Côte-Vertu / No 2308,45.505007,-73.694125,https://www.stm.info/fr/recherche#stq=55898,0,NaN,1
6334,55934,55934,Beaulac / Thimens,45.497980,-73.709809,https://www.stm.info/fr/recherche#stq=55934,0,NaN,1


In [112]:
# Display the resulting stops with latitude and longitude
new_route_stops = route_stops.drop(
    columns=['stop_code', 'stop_url', 'location_type', 'parent_station', 'wheelchair_boarding'], 
    axis=1).rename(columns={"stop_lat": "latitude", "stop_lon": "longitude"}).reset_index(drop=True)

new_route_stops.to_json

,stop_id,stop_name,latitude,longitude
0,50325,Sauvé / de l'Acadie (côté est),45.535882,-73.665750
1,50328,Sauvé / de l'Acadie,45.536011,-73.665364
2,50330,Sauvé / de l'Acadie (Côté ouest),45.535195,-73.665883
3,50341,Sauvé / Meilleur,45.540450,-73.664312
4,50342,Sauvé / Meilleur,45.540662,-73.663885
...,...,...,...,...
93,55865,Côte-Vertu / No 100,45.529531,-73.668314
94,55892,Cavendish / Place Vertu,45.500335,-73.704089
95,55898,Côte-Vertu / No 2308,45.505007,-73.694125
96,55934,Beaulac / Thimens,45.497980,-73.709809
